In [41]:
import numpy as np
import glob
import astropy
from astropy.io import fits
import os
from matplotlib import colors
import matplotlib.pyplot as plt
import casatasks
import casatools
import os
import sys
import subprocess
import tqdm
import multiprocessing as mp
import cv2

In [ ]:
data_location = "/lustre/xhall/2023-08-19_24hour_run"
file_list = np.sort(glob.glob(os.path.join(data_location, "*.ms")))

In [ ]:
image_output = "/lustre/xhall/2023-08-19_24hour_run_20min_images"

In [ ]:
len(file_list)

In [ ]:
time_spacing = 6 * 20 #20 Minutes
choosen_images = []
for i in range(int(len(file_list) / (time_spacing) + 1)):
    choosen_images.append(file_list[i * time_spacing])

In [ ]:
def make_image(file_name):
    msfile_name = file_name.split("/")[-1].split(".")
    msfile_prefix = msfile_name[0]
    image_output_new = os.path.join(image_output, msfile_prefix)
    if(not(os.path.exists(image_output_new))):
        os.mkdir(image_output_new)

    im_output = subprocess.run(["sh", "/home/xhall/GitHub/lwa-snapshot-imaging/im_maker.sh", os.path.join(image_output_new, msfile_prefix), file_name], capture_output = True, text = True)

In [ ]:
pool = mp.Pool(processes=20)
for _ in tqdm.tqdm(pool.map(make_image, choosen_images), total=len(choosen_images)):
        pass

In [ ]:
png_output = "/lustre/xhall/2023-08-19_24hour_run_20min_images/images"

In [ ]:
for file_name in choosen_images:
    msfile_name = file_name.split("/")[-1].split(".")
    msfile_prefix = msfile_name[0]
    image_output_new = os.path.join(image_output, msfile_prefix)
    fits_location = os.path.join(image_output_new, msfile_prefix + "-image.fits")
    image = fits.open(fits_location)[0].data[0][0]

    plt.figure(figsize = (12,10))
    plt.imshow(image, origin='lower', cmap='gray', vmin = 0, vmax = 70)
    plt.title(msfile_prefix)
    plt.colorbar()
    plt.savefig(os.path.join(png_output, msfile_prefix + ".png"), dpi = 300)
    plt.close()

In [43]:
video_name = '2023_08_19_24hourset_sorted.avi'

images = [img for img in np.sort(os.listdir(png_output)) if img.endswith(".png")]
frame = cv2.imread(os.path.join(png_output, images[0]))
height, width, layers = frame.shape

video = cv2.VideoWriter(os.path.join(png_output, video_name), 0, 1, (width,height))

for image in images:
    video.write(cv2.imread(os.path.join(png_output, image)))

cv2.destroyAllWindows()
video.release()